In [1]:
# ----- 環境変数読み込み -----
from dotenv import load_dotenv
load_dotenv(dotenv_path="../../../.env")

True

In [4]:
# コスト計算のためのオブジェクトを生成
import tiktoken
from llama_index.callbacks import CallbackManager, TokenCountingHandler

token_counter = TokenCountingHandler(
    tokenizer=tiktoken.encoding_for_model("gpt-3.5-turbo").encode
)

callback_manager = CallbackManager([token_counter])

In [5]:
# ----- インデックス化 -----
from llama_index import VectorStoreIndex, SimpleDirectoryReader, ServiceContext

documents = SimpleDirectoryReader('data').load_data()

service_context = ServiceContext.from_defaults(
    callback_manager=callback_manager
)
index = VectorStoreIndex.from_documents(documents, service_context=service_context)

In [8]:
print(
    "Embedding Tokens: ",
    token_counter.total_embedding_token_count,
    "\n",
    "LLM Prompt Tokens: ",
    token_counter.prompt_llm_token_count,
    "\n",
    "LLM Completion Tokens: ",
    token_counter.completion_llm_token_count,
    "\n",
    "Total LLM Token Count: ",
    token_counter.total_llm_token_count,
    "\n",
)

# reset counts
token_counter.reset_counts()

Embedding Tokens:  18 
 LLM Prompt Tokens:  804 
 LLM Completion Tokens:  91 
 Total LLM Token Count:  895 



In [7]:
# ----- インデックスを利用したQA -----
query_engine = index.as_query_engine()
response = query_engine.query("アンナは近年どのような活動をしていますか?")
print(response)


アンナは近年、世界各地でのワークショップや公演を企画しています。彼女の情熱と才能は多くの人々に感動を与え、ダンスを通じて多くの人々をつなげる架け橋となっています。


In [ ]:
# ----- インデックスの保存 -----
index.storage_context.persist(persist_dir=".data")

In [ ]:
# -----　インデックスの読み込み -----
from llama_index import StorageContext, load_index_from_storage
storage_context = StorageContext.from_defaults(persist_dir=".data")
saved_index = load_index_from_storage(storage_context)

response_from_saved_index = saved_index.as_query_engine().query("""
ディランの生い立ちを簡潔に教えてください。
""")
print(response_from_saved_index)